In [1]:
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Dropout
from keras.models import Model
import numpy as np
import keras
import pandas as pd
import handler

from __future__ import division
import matplotlib.pyplot as plt

from keras.layers.recurrent import LSTM
import time, helper
print 'complete'

Using TensorFlow backend.


ImportError: No module named helper

In [ ]:
def get_stock_data(stock_name):
    url="http://www.google.com/finance/historical?q="+stock_name+"&startdate=Jul+12%2C+2010&num=30&ei=rCtlWZGSFN3KsQHwrqWQCw&output=csv"

    df = pd.DataFrame.from_csv(url)

    return df

In [ ]:
stock_name = 'GOOGL'
df = get_stock_data(stock_name)
df = df.iloc[::-1]
print df.head()
stockArray = df.as_matrix()


In [ ]:
# split data into training, testing and validation data
x_open_train, y_open_train, x_open_valid, y_open_valid, x_open_test, y_open_test = handler.handleData(stockArray[:,0], 50, True, False)
x_high_train, y_high_train, x_high_valid, y_high_valid, x_high_test, y_high_test = handler.handleData(stockArray[:,1], 50, True, False)
x_low_train, y_low_train, x_low_valid, y_low_valid, x_low_test, y_low_test = handler.handleData(stockArray[:,2], 50, True,False)
x_vol_train, y_vol_train, x_vol_valid, y_vol_valid, x_vol_test, y_vol_test = handler.handleData(stockArray[:,4], 50, True,False)
x_orig_test, norms, x_close_train, y_close_train, x_close_valid, y_close_valid, x_close_test, y_close_test = handler.handleData(stockArray[:,3], 50, True,True)


In [ ]:
from keras import optimizers
d = 0.4


# First Input
main_input = Input(shape=(None, 1), dtype='float32', name='main_input')
x = LSTM(input_dim = 1,
               output_dim = 100,
               return_sequences = True)(main_input)
x = Dropout(d)(x)
x = LSTM(128, return_sequences = False)(x)
x = Dropout(d)(x)



# Second Input
aux_input = Input(shape=(None, 1), dtype="float32", name ="aux_input")
y = LSTM(input_dim = 1,
               output_dim = 50,
               return_sequences = True)(main_input)
y = Dropout(d)(y)
y = LSTM(100, return_sequences = False)(y)
y = Dropout(0.8)(y)



# Third Input
tert_input = Input(shape=(None, 1), dtype="float32", name ="tert_input")
a = LSTM(input_dim = 1,
               output_dim = 50,
               return_sequences = True)(main_input)
a = Dropout(d)(a)
a = LSTM(100, return_sequences = False)(a)
a = Dropout(0.8)(a)



# Concatination of inputs
z = keras.layers.concatenate([x, y])


x = Dense(128, activation="relu")(x)
x = Dropout(d)(x)
x = Dense(128,activation="relu" )(x)
x = Dropout(d)(x)
x = Dense(128, activation ='relu')(x)
x = Dense(64, activation='relu')(x)


main_output = Dense(1, activation="tanh", name ="main_output")(x)

start = time.time()
model = Model(inputs=[main_input, aux_input, tert_input], outputs=[main_output])


model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

print 'compilation time : ', time.time() - start


In [ ]:
model.fit([x_close_train, x_open_train, x_high_train], [y_close_train],
          epochs=10, batch_size=128)

In [ ]:
# Find accuracy of and plot validation data

p = model.predict([x_close_valid, x_open_valid, x_high_valid])

correct = 0
incorrect = 0

for i, item in enumerate(p):
    if i >0:
        pred = p[i] - x_close_valid[i, -1]
        truth = y_close_valid[i] - x_close_valid[i,-1]
        
        if pred/truth > 0:
            
            correct = correct + 1
            
        else:
            incorrect = incorrect + 1

prop = correct / (correct + incorrect)
print "Proportion correct is: ", prop

plt.title("Validation Data Plot")
plt.plot(p, label = "Predictions")
plt.plot(y_close_valid, label = "Actual results")
plt.legend()
plt.show()


In [ ]:
p = model.predict([x_close_test, x_open_test, x_high_test])

p_denorm = handler.denormalise_sequence(p, norms)

# Plot to normalised predictions

plt.subplot(211)
plt.title("Normalised predictions plot")
plt.plot(p, color = "green", label = "Predictions")
plt.plot(y_close_test, color = "yellow", label="Actual Data")
plt.legend(loc="upper left")


correct = 0
incorrect = 0


y_denorm = handler.denormalise_sequence(y_close_test, norms)

# Find accuracy of denormalised predictions
for i, item in enumerate(p_denorm):
    if i >0:
        pred = p_denorm[i] - y_denorm[i-1]
        truth = y_denorm[i] - y_denorm[i-1]
        
        if pred/truth > 0:
            correct = correct + 1
        else:
            incorrect = incorrect + 1

propDenorm = correct / (correct + incorrect)
print "Proportion correct is: ", prop

# Check above prediction on normalised predictions
for i, item in enumerate(p):
    if i > 0:
        pred = p[i] - x_close_test[i,-1]
        truth = y_close_test[i] - x_close_test[i,-1]
        
        if pred/truth > 0:
            
            correct = correct + 1
            
        else:
            incorrect = incorrect + 1

propNorm = correct / (correct + incorrect)
print "Proportion correct is: ", prop

if propDenorm == propNorm:
    print "Predictions match!"
    
else:
    print "Predictions do not match!"

# Plot denormalised predictions
plt.subplot(212)
plt.title("Denormalised predictictions plot")
plt.plot(p_denorm, color = "red", label = "Predictions")
plt.plot(y_denorm ,color='blue', label='Actual Data')

plt.legend(loc='upper left')
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=2.0)
plt.show()